# Operators

Operators are special keywords in SQL that we use in conjunction with SQL clauses to:
* Compare the values of fields
* Select a subset of fields
* Perform arithmetic operations

# Types of Operators

**Comparison Operators**:
1. Equal To (`=`)
2. Greater Than (`>`)
3. Less Than (`<`)
4. Greater Than or Equal To (`>=`)
5. Less Than or Equal To (`<=`)
6. Not Equal To (`<>`)

**Logical Operators**:
1. AND
2. OR
3. LIKE
4. IN
5. BETWEEN

**Arithmetic Operators**:
1. Addition
2. Subtraction
3. Division
4. Multiply
5. Modulo


# `WHERE` Clause

The `WHERE` clause allows us to add specific conditions to our queries.
* Using `WHERE`, we can limit the result to only the data that satisfies our condition.
* Filter rows in the data by running conditions.
* We can use the `WHERE` clause in conjunction with operators (Comparison, Logical, and Arithmetic).

> *Note: When using text as criteria in the WHERE clause, the text must be surrounded by SINGLE QUOTES (' '). Otherwise, it will be treated as column_name*

**SYNTAX**:
```postgresql
SELECT 
    column_list
FROM tablename
WHERE
    conditions;
    
```

**EXAMPLE**:
```postgresql
-- Get all English language movies

SELECT * FROM movies 
WHERE 
    movie_lang = 'English';

```

***Note: If you use the column aliases in the `SELECT` clause, then you cannot use the aliases in the `WHERE` clause.***

```postgresql
SELECT
    first_name,
    last_name AS surname
FROM actors
WHERE
    surname = 'Allen'; --ERROR: column "surname" does not exist.
    
```

# `AND` operator

```postgresql
-- Get all English language movies AND age certificate to 18. 

SELECT * FROM movies 
WHERE 
    movie_lang = 'English'
    AND age_certificate = '18';

-- Get all English language movies having a director ID equal to 10

SELECT * FROM movies 
WHERE 
    movie_lang = 'English'
    AND director_id = '10';
```

# `OR` Operator

```postgresql
-- Get all English Language OR Chinese movies

SELECT * FROM movies
WHERE
    movie_lang = 'English'
    OR movie_lang = 'Chinese';

```

# Execution Order of `AND`, `OR` operators

* The `AND` operator is processed first, and then the `OR` operator is processed next.
* SQL processes the `AND` operator like **multiplication** and the `OR` operator like **addition**, unless you include parentheses to specify the **Order of Execution**.
* Without parentheses, the `AND` operator will be processed in the same way as `1+2*3 = 7`, but `(1+2)*3 = 9`.
* It is ALWAYS better to use parentheses whenever you're using multiple operators so that the **Order of Operation** is clearly defined.

**EXAMPLE**:
```postgresql
-- Get all English Language OR Chinese movies AND movies with age_certification equal to 12

SELECT * FROM movies
WHERE
    movie_lang = 'English'
    OR movie_lang = 'Chinese'
    AND age_certificate = '12';  
    
-- But here, you get Chinese movies whose age_certificate is NOT EQUAL TO '12'
-- Because, without parentheses, PostgreSQL evaluates the AND operator first, then the OR operator.
-- That is, (movie_lang = 'English' AND age_certificate = '12') OR movie_lang = 'Chinese'.
-- Hence, we get Chinese movies whose age_certificate is NOT EQUAL TO '12' in the result too.

-- With parentheses - The correct way!!!

SELECT * FROM movies
WHERE
    (movie_lang = 'English'
    OR movie_lang = 'Chinese')
    AND age_certificate = '12';    
```

> ***Order of Parentheses & Operators matters.***

# Order of execution of `WHERE`, `SELECT` & `ORDER BY` clauses

**What is the order of execution of the `WHERE` clause?**

PostgreSQL evaluates the WHERE clause:
* after the `FROM` clause, and
* before the `SELECT` and `ORDER BY` clause.

That is, **`FROM` > `WHERE` > `SELECT` > `ORDER BY`**.

**EXAMPLE**:

```postgresql
SELECT * FROM movies
WHERE
    movie_lang = 'English'
ORDER BY 
    movie_length DESC;
    
```

# Using Comparison Operators

```postgresql
-- 1. Get all movies where the movie length is greater than 100
SELECT * FROM movies
WHERE movie_length > 100;
ORDER BY movie_length;

-- 2. Get all movies where the movie length is greater than or equal to 100
SELECT * FROM movies
WHERE movie_length >= 100;
ORDER BY movie_length;

-- 3. Get all movies where the movie length is less than 100
SELECT * FROM movies
WHERE movie_length < 100;
ORDER BY movie_length;

-- 4. Get all movies where the movie length is less than or equal to 100
SELECT * FROM movies
WHERE movie_length <= 100;
ORDER BY movie_length;

-- 5. Get all movies which is not in the english language
SELECT * FROM movies
WHERE movie_lang <> 'English';

SELECT * FROM movies
WHERE movie_lang != 'English';

```

**Can we work with date data types?**

When querying for dates, it is important first to take a look at how the date is stored in the table that you're querying. That is, `YYY-MM-DD` or `DD-MM-YYYY`, etc.

```postgresql
-- Get all movies where the release date is greater than 2000

SELECT * FROM movies
WHERE release_date > '2000-01-01' -- Notice the quotes used for date value and format is YYYY-MM-DD
ORDER BY release_date ASC;

```

# Using `LIMIT` and `OFFSET`

Using LIMIT to limit the number of output records.

**SYNTAX**:
```postgresql
SELECT
    column_list
FROM tablename
ORDER BY columnname
LIMIT number;

```

**EXAMPLE**:
```postgresql
-- 1. Get the top 5 biggest movies by movie length
SELECT * FROM
ORDER BY movies_length DESC
LIMIT 5;

-- 2. Get the top 5 oldest American directors
SELECT * FROM directors
WHERE nationality = 'American'
ORDER BY date_of_birth ASC
LIMIT 5;

-- 3. Get the top 10 most domestically profitable movies
SELECT * FROM movies_revenues
ORDER BY revenues_domestic DESC NULLS LAST
LIMIT 10;

-- 5. List 5 films starting from the fourth one, ordered by movie_id
SELECT * FROM movies
ORDER BY movie_id DESC NULLS LAST
LIMIT 5 OFFSET 4; 

-- OFFSET is exclusive. 
-- That is, the number specified with OFFSET is excluded from the result. 
-- Here are the TOP 5 records from the 5th Records onwards (i.e. OFFSET 4 excludes the first 4 records). 

```

# Using `FETCH` 

* `FETCH` clause is used to retrieve a portion of the rows returned by a query.
* `FETCH` clause was introduced in SQL 2008.
* `FETCH` clause functionality is equivalent to the LIMIT clause.
* It is recommended to use the `FETCH`clause because it follows the standard ANSI SQL.

**SYNTAX**:
```postgresql
FETCH {FIRST | NEXT} {row_count} {ROW | ROWS} ONLY
OFFSET start {ROW | ROWS}
```

The `OFFSET start` is an integer that must be zero or positive.
* By default, `start` is 0.
* In case the `start` is greater than the number of rows in the result set, no rows are returned.

**EXAMPLE**:
```postgresql
-- 1. Get the first row of the movies table
SELECT * FROM movies
FETCH FIRST 1 ROW ONLY;

-- 2. Get the top 5 biggest movies by movie length
SELECT * FROM movies
ORDER BY movie_length DESC
FETCH FIRST 5 ROWS ONLY;

-- 3. Get the top 5 oldest American directors
SELECT * FROM directors
WHERE nationality = 'American'
ORDER BY date_of_birth ASC
FETCH FIRST 5 ROWS ONLY;

-- 4. Get the first 5 movies from the 5th record onwards by long movie length
SELECT * FROM movies
ORDER BY movie_length DESC
FETCH FIRST 5 ROWS ONLY
OFFSET 5;
```

# Using `IN` and `NOT IN` operator

The `IN` and `NOT IN` operators are used to check if a value matches or does not match in a list.
* The `IN` operator returns `true` if a match is found, otherwise returns `false`.
* The `NOT IN` operator returns `true` if a match is **NOT** found, otherwise returns `false`.

```postgresql
-- 1. Get all movies for the English, Chinese, and Japanese languages

SELECT * FROM movies
WHERE 
    movie_lang = 'English'
    OR movie_lang = 'Chinese'
    OR movie_lang = 'Japanese';     -- if we had 10 more criteria, then we would have to write 10 more such conditions. This is where the IN and NOT IN operators make it simpler and easier.
ORDER BY movie_lang;

-- Using IN Operator

SELECT * FROM movies
WHERE 
    movie_lang IN ('English', 'Chinese', 'Japanese')
ORDER BY movie_lang;

-- Using NOT IN Operator

SELECT * FROM movies
WHERE 
    movie_lang NOT IN ('English', 'Chinese', 'Japanese')
ORDER BY movie_lang;

-- 2. Get all movies where age_certification is 12 and PG type
SELECT * FROM movies
WHERE 
    age_certificate IN ('12', 'PG')
ORDER BY age_certificate ASC;

-- 3. Get all movies where the director ID is not 13 or 10
SELECT * FROM movies
WHERE 
    director_id NOT IN (13, 10)
ORDER BY director_id ASC;

```

# Using `BETWEEN` and `NOT BETWEEN`

The `BETWEEN` operator matches a value against a range of values.

**SYNTAX**:
```postgresql
value BETWEEN low_range and high_range
```
* If the value is **greater than or equal to** the `low_range` and **less than or equal to** the `high_range`, the expression returns `true`, otherwise, it returns `false`.
* Both `low_range` and `high_range` values are **inclusive**.

**EXAMPLE**:

```postgresql
-- 1. Get all actors where birth_date is between 1991 and 1995
SELECT * FROM actors
WHERE 
    date_of_birth BETWEEN '1991-01-01' AND '1995-01-01'
ORDER BY date_of_birth ASC;

-- 2. Get all movies where domestic revenues are between 102.10 and 290.30 (inclusive)
SELECT * FROM movies_revenues
WHERE
    revenues_domestic BETWEEN 102.10 AND 290.30
ORDER BY revenues_domestic ASC;

-- 3. Get all movies where domestic revenues are NOT between 102.10 and 290.30 (inclusive)
SELECT * FROM movies_revenues
WHERE
    revenues_domestic NOT BETWEEN 102.10 AND 290.30
ORDER BY revenues_domestic ASC;

```